## 1. Data Exploration

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file into a DataFrame with low_memory=False to avoid DtypeWarning
data = pd.read_csv('msna_data.csv', low_memory=False)

# Explore the data
print("\nInformation about the DataFrame:")
print(data.info())

print("\nSummary statistics of the DataFrame:")
print(data.describe(include='all'))

# Display the data types of each column
print("\nData types of each column:")
print(data.dtypes)

## 2. Feature Engineering

### 2.1 Data Cleaning

The data cleaning process begins by removing rows with missing values in the target column, `child_sep_severity`. Next, only the relevant variables from the `vars_analysis.csv` file are retained for further analysis. Columns with more than 10% missing values are dropped to ensure data quality. Additionally, columns with zero variance, which provide no useful information, are removed. Variables ending in '_NA' without corresponding non-NA versions are also discarded to avoid redundancy. Missing data patterns are visualized both before and after these cleaning steps. Finally, the cleaned dataset is reviewed to ensure all essential variables remain, with any dropped variables from `vars_analysis` reported.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno

# Define function to display and drop columns based on a condition
def drop_columns_with_condition(df, condition, description):
    columns_to_drop = df.columns[condition].tolist()
    if columns_to_drop:
        print(f"\nDropped columns {description}:")
        print(columns_to_drop)
        return df.drop(columns=columns_to_drop)
    return df

# Filter out rows with missing values in the 'child_sep_severity' column
data_filtered = data.dropna(subset=['child_sep_severity'])
print("\nShape of the filtered DataFrame, without missing values for the response variable:", data_filtered.shape)

# Load variables to consider for the model
vars_analysis = pd.read_csv('vars_analysis.csv')['variable'].tolist()

# Select columns that are relevant for analysis
data_select = data_filtered[vars_analysis].copy()
print("\nDataFrame after keeping variables in vars_analysis:")
data_select.info()

# Identify columns with more than 20% missing values
missing_percentage = data_select.isnull().mean() * 100
columns_with_many_missing = missing_percentage > 20

# Drop columns with more than 20% missing values
data_select = drop_columns_with_condition(data_select, columns_with_many_missing, "with more than 20% missing values")

# Drop columns with zero variance (only one unique value)
zero_variance_condition = data_select.nunique() <= 1
data_select = drop_columns_with_condition(data_select, zero_variance_condition, "with zero variance")

# Identify and drop variables ending with '_NA' that have no other variable with the same stem
na_columns = [col for col in data_select.columns if col.endswith('_NA')]
stems = {col[:-3] for col in na_columns}
na_suffix_condition = [col for col in na_columns if not any(other_col.startswith(col[:-3]) and other_col != col for other_col in data_select.columns)]
data_select = drop_columns_with_condition(data_select, data_select.columns.isin(na_suffix_condition), "'_NA' with no matching stem variable")

# Assign the final cleaned data to data_select_na
data_select_na = data_select

# Visualize missing data matrix before dropping columns
msno.matrix(data_filtered)
plt.show()

# Visualize missing data matrix after dropping columns
msno.matrix(data_select_na)
plt.show()

# Display final DataFrame info
print("\nFinal DataFrame after dropping unnecessary columns:")
data_select_na.info()

# Check and display which variables from vars_analysis are no longer in the data
missing_vars = [var for var in vars_analysis if var not in data_select_na.columns]
print("\nVariables from vars_analysis no longer in the data:", missing_vars)


### 2.2 Change Encoding of Ordinal Variables

The script converts one-hot encoded ordinal variables into a more appropriate label-encoded format. It begins by loading a mapping of old one-hot labels and new ordinal labels from `new_labels.csv`, filtering for variables present in the dataset. The script then replaces one-hot encoded values with the corresponding ordinal labels and converts the recoded variables to integer type, ensuring proper handling of missing values. It verifies that all required variables have been recoded and displays the updated data along with the data types of the recoded variables.


In [ ]:
# Load the new_labels CSV file and filter for variables present in the data
new_labels = pd.read_csv('new_labels.csv', usecols=['variable', 'label_old', 'label_new'])
new_labels = new_labels[new_labels['variable'].isin(data_select_na.columns)]

print("\nLoaded new_labels DataFrame:")
print(new_labels.head())

# Create a copy of data_select_na to apply changes
data_ord = data_select_na.copy()

# Apply the label mapping to the data_ord DataFrame
for _, row in new_labels.iterrows():
    variable = row['variable']
    label_old = row['label_old']
    label_new = int(row['label_new'])  # Convert label_new to integer
    
    if variable in data_ord.columns:
        data_ord[variable] = data_ord[variable].replace(label_old, label_new)

# Define recoded_variables
recoded_variables = new_labels['variable'].unique()

# Convert the recoded variables to integer type
for variable in recoded_variables:
    if variable in data_ord.columns:
        data_ord[variable] = data_ord[variable].fillna(pd.NA).astype('Int64')  # Fill NaNs with pd.NA before converting

# Check if recoded_variables are in data_ord
variables_in_data_ord = all(variable in data_ord.columns for variable in recoded_variables)
print(f"All recoded variables are in data_ord: {variables_in_data_ord}")

# Identify which variables are not in data_ord
missing_variables = [variable for variable in recoded_variables if variable not in data_ord.columns]
print(f"Variables not in data_ord: {missing_variables}")

# Display the first few rows of the recoded variables in the data_ord DataFrame
print("\nRecoded variables in data_ord DataFrame:")
print(data_ord[recoded_variables].head())

# Display the variable types of the recoded variables
print("\nVariable types of recoded variables:")
print(data_ord[recoded_variables].dtypes)

### 2.4 KNN Imputation of Remaining Missing Values

Next, we use **K-Nearest Neighbors (KNN) imputation** to handle missing values across all columns in the dataset. It first applies the `KNNImputer` to estimate and replace missing values based on the nearest neighbors. After imputation, the missing data is visualized, and the updated dataset with the imputed values is displayed for verification.

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
import missingno as msno
import matplotlib.pyplot as plt

# Function to apply K-Nearest Neighbors Imputation to all columns
def knn_impute_all_columns(df, n_neighbors=5):
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    return df_imputed

# Apply KNN imputation to all columns
data_na_imp = knn_impute_all_columns(data_ord.copy())

# Visualize missing data matrix
msno.matrix(data_na_imp)
plt.show()

# Display the first few rows of the new dataframe with imputed values
print("\nDataFrame with imputed values in all columns:")
print(data_na_imp.head())

# Display the first few rows of the new dataframe with imputed values
print("\nDataFrame with imputed values in the first column:")
print(data_na_imp.head())

### 2.5 Standardization

This process standardizes the **numerical variables** in the dataset while preserving categorical and dummy variables. After identifying variable types from `vars_analysis.csv`, only numerical variables are standardized using `StandardScaler` to ensure a mean of 0 and standard deviation of 1. Categorical variables are added back without standardization. The script displays summary statistics before and after standardization and visualizes the distribution of the `num_children` variable with histograms to illustrate the effect of standardization.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

# Load the vars_analysis.csv to identify variable types
vars_analysis = pd.read_csv('vars_analysis.csv', usecols=['variable', 'type'])
dummy_variables = vars_analysis[vars_analysis['type'] == 'dummy']['variable'].tolist()
categorical_variables = vars_analysis[vars_analysis['type'] == 'ordinal']['variable'].tolist()
numerical_variables = vars_analysis[vars_analysis['type'] == 'numerical']['variable'].tolist()

# Filter variables to include only those present in data_dum
dummy_variables = [var for var in dummy_variables if var in data_na_imp.columns]
categorical_variables = [var for var in categorical_variables if var in data_na_imp.columns]
numerical_variables = [var for var in numerical_variables if var in data_na_imp.columns]

# Standardize the numerical variables (excluding dummy variables and categorical variables)
scaler = StandardScaler()
data_stand = data_na_imp.copy()
data_stand[numerical_variables] = scaler.fit_transform(data_stand[numerical_variables])

# Add back the categorical variables without standardizing them
common_categorical_vars = list(set(categorical_variables).intersection(data_na_imp.columns))
data_stand[common_categorical_vars] = data_na_imp[common_categorical_vars]

# Display summary statistics before standardization
print("\nSummary statistics before standardization:")
print(data_na_imp.describe())

# Display summary statistics after standardization
print("\nSummary statistics after standardization:")
print(data_stand.describe())

# Histogram for hoh_age before standardization
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(data_na_imp['num_children'], bins=30, alpha=0.7, color='blue')
plt.title('num_children before standardization')
plt.xlabel('num_children')
plt.ylabel('Frequency')

# Histogram for num_children after standardization
plt.subplot(1, 2, 2)
plt.hist(data_stand['num_children'], bins=30, alpha=0.7, color='green')
plt.title('num_children after standardization')
plt.xlabel('num_children')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 3. Feature Selection

This process performs **feature selection** using various methods to identify the most important features for predicting the target variable. The dataset is first prepared by ensuring all features are numeric, free of missing values, and standardized. Several feature selection methods are applied:
1. **Recursive Feature Elimination (RFE)**: Selects features by recursively removing the least important ones based on a logistic regression model.
2. **Random Forest Feature Importance**: Identifies the top features based on the importance scores from a Random Forest model.
3. **Elastic Net Regularization**: Selects features based on the non-zero coefficients from an Elastic Net model.
4. **Boruta Algorithm**: A wrapper method using Random Forest to identify all relevant features.
5. **Mutual Information**: Measures the dependency between each feature and the target.

Finally, the selected features from all methods are combined, and the dataset is filtered to retain only those features, preparing it for further analysis or modeling.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from boruta import BorutaPy
import shap

# Assuming data_stand is already defined and loaded

# Create a copy of data_stand for feature selection
data_fs = data_stand.copy()

# Separate features and target variable
X = data_fs.drop(columns=['child_sep_severity_family_reun_required'])
y = data_fs['child_sep_severity_family_reun_required']

### Convert All Variables to Float ###

# Force all features to float64
X = X.astype(np.float64)
y = y.astype(np.float64)

### Utility Functions for Checks ###

# Check if all columns are numeric
def check_numeric(df):
    if df.select_dtypes(include=[np.number]).shape[1] != df.shape[1]:
        raise ValueError("All features must be numeric. Please convert categorical variables to numeric format.")

# Check for missing values
def check_no_missing(df):
    if df.isnull().sum().sum() > 0:
        raise ValueError("Input data contains missing values. Please handle missing data before applying these methods.")

# Ensure target variable is binary or multiclass
def check_target(y):
    if not pd.api.types.is_numeric_dtype(y):
        raise ValueError("Target variable 'y' must be numeric.")
    if len(y.unique()) < 2:
        raise ValueError("Target variable 'y' must have at least two classes for classification.")

# Check if data is standardized and print variables that are not
def check_standardized(df, mean_threshold=0.01, std_threshold=0.01):
    # Calculate the absolute difference of means from 0 and standard deviations from 1
    mean_diff = df.mean().abs()  # Difference from 0 for means
    std_diff = (df.std() - 1).abs()  # Difference from 1 for standard deviations

    # Identify columns that are not standardized
    non_standardized_vars = df.columns[(mean_diff > mean_threshold) | (std_diff > std_threshold)].tolist()
    
    # Print warning only if there are non-standardized variables
    if non_standardized_vars:
        print("Warning: The following variables do not appear to be standardized (mean ≈ 0, std ≈ 1):")
        print(non_standardized_vars)

### 1. Ensure Input Data is Correct for Each Method ###

# Check target variable
check_target(y)

# Check if features are numeric and have no missing values
check_numeric(X)
check_no_missing(X)

### 2. Check Standardization for Methods Sensitive to Scale ###
check_standardized(X)

### 3. Recursive Feature Elimination (RFE) ###
try:
    print("Running RFE...")
    model = LogisticRegression(max_iter=1000)
    rfe = RFE(model, n_features_to_select=10)
    X_rfe = rfe.fit_transform(X, y)
    print(f"Features after RFE: {X_rfe.shape[1]}")
except Exception as e:
    print(f"Error running RFE: {e}")

### 4. Feature Importance from Tree-Based Models ###
try:
    print("Running Random Forest...")
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X, y)

    # Get feature importances
    importances = rf.feature_importances_
    indices = np.argsort(importances)[::-1]
    top_features_rf = X.columns[indices[:10]]
    print(f"Top features according to Random Forest: {top_features_rf}")
except Exception as e:
    print(f"Error running Random Forest: {e}")

### 5. Elastic Net Regularization ###
try:
    print("Running Elastic Net...")
    elastic_net = ElasticNetCV(cv=5, random_state=42)
    elastic_net.fit(X, y)
    enet_coef = pd.Series(elastic_net.coef_, index=X.columns)
    top_features_enet = enet_coef[enet_coef != 0].index
    print(f"Top features according to Elastic Net: {top_features_enet}")
except Exception as e:
    print(f"Error running Elastic Net: {e}")

### 6. Boruta Algorithm ###
try:
    print("Running Boruta...")
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42)
    boruta_selector.fit(X.values, y)
    boruta_features = X.columns[boruta_selector.support_].tolist()
    print(f"Top features according to Boruta: {boruta_features}")
except Exception as e:
    print(f"Error running Boruta: {e}")

### 7. Mutual Information ###
try:
    print("Calculating Mutual Information...")
    mi = mutual_info_classif(X, y)
    mi_features = X.columns[np.argsort(mi)[-10:]].tolist()
    print(f"Top features according to Mutual Information: {mi_features}")
except Exception as e:
    print(f"Error calculating Mutual Information: {e}")

### Combine Results for the Most Relevant Features ###
try:
    combined_features = set(top_features_rf) | set(top_features_enet) | set(boruta_features) | set(mi_features)
    print(f"Combined selected features: {combined_features}")
except Exception as e:
    print(f"Error combining selected features: {e}")

### Drop Unselected Features ###

# Find the intersection of selected features from all methods
selected_features = list(combined_features)  # Convert set to list for indexing

# Filter the original dataset to keep only the selected features
X_selected = X[selected_features]

# Print the shape of the new dataset to confirm feature selection
print(f"Shape of data after dropping unselected features: {X_selected.shape}")

## 4. Data Splitting

Next, we split the dataset into **training**, **validation**, and **test** sets while maintaining the class distribution due to the highly imbalanced nature of the target variable.

1. **Initial Split**: The dataset is first divided into an 80% **train + validation** set and a 20% **test set**.
2. **Train/Validation Split**: The train + validation set is further split into a 75% **training set** and a 25% **validation set**.
   - This results in:
     - **Training Set**: 60% of the original data
     - **Validation Set**: 20% of the original data
     - **Test Set**: 20% of the original data
3. **Class Distribution**: After splitting, the number of samples and the class distribution (for both classes 0 and 1) are printed for each dataset to ensure the splits maintain the correct proportions of each class.

This ensures proper stratified splitting of the dataset for training, hyperparameter tuning, and final testing.

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming X is your feature set and y is your highly imbalanced response variable
# First, create a train + validation set and a test set (e.g., 80% train/validation, 20% test)
X_train_val, X_test, y_train_val, y_test = train_test_split(X_selected, y, test_size=0.2, stratify=y, random_state=42)

# Then, split the train + validation set into the actual training set and validation set (e.g., 75% train, 25% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42)

# Result:
# - X_train, y_train: Training data (60% of the original dataset)
# - X_val, y_val: Validation data (20% of the original dataset)
# - X_test, y_test: Test data (20% of the original dataset)

# Print the structure of the resulting splits
print("Training Set")
print(f"  Features: {X_train.shape[1]}")
print(f"  Samples: {X_train.shape[0]}")
print(f"  Class 0: {np.sum(y_train == 0)}")
print(f"  Class 1: {np.sum(y_train == 1)}")
print("\nValidation Set")
print(f"  Features: {X_val.shape[1]}")
print(f"  Samples: {X_val.shape[0]}")
print(f"  Class 0: {np.sum(y_val == 0)}")
print(f"  Class 1: {np.sum(y_val == 1)}")
print("\nTest Set")
print(f"  Features: {X_test.shape[1]}")
print(f"  Samples: {X_test.shape[0]}")
print(f"  Class 0: {np.sum(y_test == 0)}")
print(f"  Class 1: {np.sum(y_test == 1)}")

## 5. Data Balancing

This script applies **SMOTEENN (Synthetic Minority Over-sampling Technique + Edited Nearest Neighbors)** to balance the training data and ensure feature consistency between the training, validation, and test sets.

1. **SMOTEENN**: Balances the training data by oversampling the minority class and removing noisy samples from the majority class, resulting in `X_train_resampled` and `y_train_resampled`.
2. **Column Check (Before Reindexing)**: The script checks for any missing or extra columns in `X_val` and `X_test` compared to `X_train_resampled`, identifying any discrepancies in feature alignment.
3. **Reindexing**: Validation and test sets are reindexed to ensure they have the same columns as `X_train_resampled`, with missing columns filled with zeros.
4. **Column Check (After Reindexing)**: A second check confirms that the feature columns in `X_val` and `X_test` now match the resampled training set.
5. **Class Distribution**: The script displays the original and resampled class distributions, illustrating the effect of SMOTEENN on balancing the dataset.

This step ensures that the resampled training data is balanced and that the validation and test sets are aligned with the training data for consistent model evaluation.

In [ ]:
from imblearn.combine import SMOTEENN
from collections import Counter

# Apply SMOTEENN to balance the training data
smote_enn = SMOTEENN(random_state=42)
X_train_resampled, y_train_resampled = smote_enn.fit_resample(X_train, y_train)

# Check for column mismatches before reindexing
print("Checking for column mismatches before reindexing...")

# Find columns missing in X_val or X_test
missing_in_val = set(X_train_resampled.columns) - set(X_val.columns)
missing_in_test = set(X_train_resampled.columns) - set(X_test.columns)

# Find extra columns in X_val or X_test
extra_in_val = set(X_val.columns) - set(X_train_resampled.columns)
extra_in_test = set(X_test.columns) - set(X_train_resampled.columns)

print(f"Columns missing in X_val: {missing_in_val}")
print(f"Columns missing in X_test: {missing_in_test}")
print(f"Extra columns in X_val: {extra_in_val}")
print(f"Extra columns in X_test: {extra_in_test}")

# Reindex the validation and test sets to match the training set
X_val = X_val.reindex(columns=X_train_resampled.columns, fill_value=0)
X_test = X_test.reindex(columns=X_train_resampled.columns, fill_value=0)

# Check again for column mismatches after reindexing
print("\nChecking for column mismatches after reindexing...")

# Find columns missing in X_val or X_test (there should be none now)
missing_in_val_after = set(X_train_resampled.columns) - set(X_val.columns)
missing_in_test_after = set(X_train_resampled.columns) - set(X_test.columns)

# Find extra columns in X_val or X_test (there should be none now)
extra_in_val_after = set(X_val.columns) - set(X_train_resampled.columns)
extra_in_test_after = set(X_test.columns) - set(X_train_resampled.columns)

print(f"Columns missing in X_val after reindexing: {missing_in_val_after}")
print(f"Columns missing in X_test after reindexing: {missing_in_test_after}")
print(f"Extra columns in X_val after reindexing: {extra_in_val_after}")
print(f"Extra columns in X_test after reindexing: {extra_in_test_after}")

# Print class distribution after resampling
print(f"\nOriginal class distribution: {Counter(y_train)}")
print(f"Resampled class distribution: {Counter(y_train_resampled)}")

## 6. Model Selection

This process uses **GridSearchCV** to tune hyperparameters for three models—**Random Forest**, **Logistic Regression**, and **XGBoost**—while addressing class imbalance. These models are chosen because:
- **Random Forest** is robust to overfitting and can handle non-linear relationships, making it effective for imbalanced data.
- **Logistic Regression** provides a simple, interpretable model and can be adjusted with class weighting to handle imbalances.
- **XGBoost** is a powerful boosting algorithm that excels at complex datasets and imbalanced data with the right hyperparameters.

The models are configured with:
- **Class Weighting** for Random Forest and Logistic Regression (`class_weight='balanced'`) to give more weight to the minority class.
- **Scale Positive Weight** for XGBoost (`scale_pos_weight`) to adjust the weight of the minority class.

The models are evaluated using the **F1-Score** as the scoring metric in GridSearchCV to focus on optimizing performance for the minority class by balancing precision and recall.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# Define candidate models with class weighting for imbalance handling
models = {
    'Random Forest': RandomForestClassifier(class_weight='balanced', random_state=42),
    'Logistic Regression': LogisticRegression(class_weight='balanced', max_iter=1000),
    'XGBoost': XGBClassifier(scale_pos_weight=len(y_train_resampled[y_train_resampled == 0]) / len(y_train_resampled[y_train_resampled == 1]), random_state=42)
}

# Define hyperparameters for GridSearch
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10],
    },
    'XGBoost': {
        'learning_rate': [0.01, 0.1],
        'n_estimators': [100, 200],
        'max_depth': [3, 5, 7]
    }
}

# Use GridSearchCV to find the best model and hyperparameters with F1-Score as the evaluation metric
best_models = {}
for model_name, model in models.items():
    print(f"Tuning {model_name}...")
    # Switch from roc_auc to f1 to better handle minority class performance
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train_resampled, y_train_resampled)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best params for {model_name}: {grid_search.best_params_}")


## 7. Model Evaluation

We now evaluate the performance of multiple trained models on a **validation set** to understand how well they generalize to unseen data.

For each model:
- We use **ROC-AUC** to measure its ability to distinguish between positive and negative classes, which is crucial for imbalanced datasets.
- We calculate the **F1-Score** to assess the balance between precision and recall, providing insight into the model’s overall classification effectiveness.
- A **classification report** is generated to give detailed metrics, including precision, recall, and F1-score for both the majority and minority classes.

By comparing these metrics across models, we aim to select the one that performs best, generalizes well, and effectively handles class imbalance.


In [ ]:
from sklearn.metrics import roc_auc_score, f1_score, classification_report

for model_name, model in best_models.items():
    print(f"\nEvaluating {model_name} on validation set:")
    y_val_pred = model.predict(X_val)
    y_val_prob = model.predict_proba(X_val)[:, 1]
    
    print(f"ROC-AUC: {roc_auc_score(y_val, y_val_prob)}")
    print(f"F1-Score: {f1_score(y_val, y_val_pred)}")
    print("\nClassification Report:")
    print(classification_report(y_val, y_val_pred))


## 8. Model Retraining

Once the best model is selected based on its performance on the validation set, we retrain it using the **full resampled training dataset** to take advantage of all the available data.

In this step:
- We identify the best model by comparing their **ROC-AUC** scores from the validation phase.
- The chosen model is then retrained using the entire **balanced training set**, which was resampled using techniques like **SMOTE + ENN** to address class imbalance.
- By using all the available training data, we ensure that the model has the maximum amount of information for learning, leading to better generalization during final testing.

This step prepares the model for its final evaluation on the test set.

In [ ]:
from sklearn.metrics import f1_score

# Select the best model based on F1-Score for the minority class
best_model_name = max(best_models, key=lambda name: f1_score(y_val, best_models[name].predict(X_val)))

best_model = best_models[best_model_name]

print(f"Retraining the best model: {best_model_name}...")
best_model.fit(X_train_resampled, y_train_resampled)


## 9. Model Evaluation on Test Set

After retraining the best model on the full resampled training data, we evaluate its performance on the **test set**, which was kept aside during the initial data splitting.

In this step:
- We use the **test set** (previously unseen by the model) to generate predictions and assess how well the model generalizes to completely new data.
- **ROC-AUC** is computed to measure how well the model distinguishes between positive and negative classes in this final evaluation.
- The **F1-Score** is calculated to evaluate the balance between precision and recall, which remains important given the imbalanced nature of the dataset.
- A **classification report** provides a detailed breakdown of precision, recall, and F1-scores for both classes, allowing us to fully understand the model's performance on real-world data.

This final evaluation gives an unbiased estimate of the model's performance before deployment.


In [ ]:
# Make predictions on the test set using the best model
y_test_pred = best_model.predict(X_test)

# Ensure that predict_proba works for XGBClassifier and get the probabilities for the positive class
y_test_prob = best_model.predict_proba(X_test)[:, 1]

# Evaluate the performance of the best model on the test set
print(f"\nTest Set Performance for {best_model_name}:")
print(f"ROC-AUC Score: {roc_auc_score(y_test, y_test_prob)}")
print(f"F1-Score: {f1_score(y_test, y_test_pred)}")

# Print the classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))


### Interpretation of XGBoost Test Set Performance:

#### 1. **ROC-AUC Score: 0.777**
This score indicates a **moderate ability** to distinguish between the two classes (0 and 1). With a score of 0.777, the model is somewhat good at ranking class 1 instances higher than class 0, but it's not particularly strong.

#### 2. **F1-Score: 0.070**
The **F1-Score** is very low, showing that the model struggles to balance precision and recall for the minority class (class 1). This suggests poor performance in detecting true positives while avoiding false positives.

#### 3. **Classification Report**:
- **Class 0 (Majority Class)**:
  - **Precision: 1.00**: The model almost perfectly identifies class 0, with nearly all predictions of class 0 being correct.
  - **Recall: 0.98**: It captures 98% of all class 0 instances, missing only a small percentage.
  - **F1-Score: 0.99**: High precision and recall lead to a very strong F1-score for class 0.

- **Class 1 (Minority Class)**:
  - **Precision: 0.04**: Only 4% of the predictions for class 1 are correct, meaning most of them are false positives.
  - **Recall: 0.25**: The model captures 25% of actual class 1 instances, missing 75%.
  - **F1-Score: 0.07**: Low precision and recall result in a poor F1-score for class 1, indicating significant difficulty in identifying the minority class.

#### 4. **Accuracy: 0.97**
While the overall accuracy is 97%, this is **misleading** due to the class imbalance. The high accuracy is mainly driven by the model's performance on class 0, not its ability to detect class 1.

#### 5. **Macro Average**:
- **Precision: 0.52**, **Recall: 0.61**, **F1-Score: 0.53**: These averages show that while the model performs well on class 0, the poor performance on class 1 lowers the overall averages.

#### 6. **Weighted Average**:
- The **weighted averages** reflect the dominance of class 0 in the dataset. The high weighted precision (0.99), recall (0.97), and F1-score (0.98) are largely influenced by the model's success in predicting class 0, masking the poor performance on class 1.

#### Conclusion:
- **Strong performance on Class 0**: The model excels at predicting class 0, with near-perfect precision and recall.
- **Weak performance on Class 1**: It struggles significantly with class 1, having very low precision (0.04) and recall (0.25), meaning most class 1 predictions are incorrect.
- **Overall**: Despite the high accuracy and ROC-AUC score, the model is ineffective at identifying the minority class. Further improvements, such
